In [40]:
import sys
sys.path.append('../src/')

from core import extract_text_from_pdf

In [1]:
# from enum import Enum

from pydantic import BaseModel

import sglang as sgl
from sglang.srt.constrained import build_regex_from_object

In [47]:
from glob import glob
from openai import OpenAI

In [2]:
from dotenv import load_dotenv
import os

import json

In [3]:
load_dotenv()

True

In [4]:
sgl.set_default_backend(sgl.RuntimeEndpoint(os.environ['LLM_HOST'][:-3]))

### Random test

In [32]:
class Answer(BaseModel):
    a: int
    b: int
    result: int


@sgl.function
def sum_gen(s, prompt):
    s += prompt
    s += "Give me a sum of two provided numbers in the JSON format.\n"
    s += sgl.gen(
        "json_output",
        max_tokens=512,
        temperature=0,
        regex=build_regex_from_object(Answer),
    )


def driver_sum_gen(prompt: str = ""):
    state = sum_gen.run(prompt)
    return state.text()

In [37]:
resp = driver_sum_gen("2, 30\n")

print(resp)

2, 30
Give me a sum of two provided numbers in the JSON format.
{ "a" : 2, "b" : 30, "result" : 32 }


### CV test

In [48]:
client = OpenAI(
    api_key=os.environ['OPENAI_API_KEY'],
    base_url=os.environ['LLM_HOST']
)

model_name = client.models.list().data[0].id

def gen(prompt: str):
    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                ],
            }
        ],
        max_tokens=2048,
        temperature=0.0,
    )
    
    return response

In [136]:
scale = """
1. Work experience:
- < 20: Less than a year
- 20: 1 years
- 50: 3 years
- 75: 6 years
- > 90: 10+ years

2. Responsibility:
- < 20: Very low (cannot delegate tasks)
- < 40: Low (needs strong supervision)
- 50: Average (needs to check periodically)
- > 75: High (sometimes better to check)
- > 90: Very high (can be relied upon)

3. Creativity:
- < 20: Very low (follows instructions)
- 30: Low (makes small suggestions)
- 50: Average (sometimes generates new ideas)
- > 75: High (often suggests innovative solutions)
- > 90: Very high (constantly suggests innovative solutions)

4. Communication skills:
- 25: Very low (difficult to communicate)
- 40: Low (limited communication)
- 50: Average (normal communication)
- 70: High (good communication)
- > 90: Very high (exceptional communication skills)

5. Leadership skills:
- < 15: Very low (never takes leadership)
- 25: Low (rarely takes leadership)
- 50: Average (sometimes takes leadership)
- 60: High (often takes leadership)
- > 80: Very high (constantly shows leadership)

6. Stress resistance:
- < 25: Very low (cannot cope with stress)
- 40: Low (difficulty coping with stress)
- 50: Average (has problems with stress)
- 70: High (no problems with stress)
- > 80: Very high (exceptional coping with stress)

7. Job difficulty:
- < 15: Very low (easy job, requiring no skills)
- 25: Low (job with basic requirements)
- 50: Average (requires some skills and knowledge)
- 75: High (requires significant knowledge and skills)
- > 90: Very high (expert level, complex tasks)

8. Average length of employment in one place:
- < 30: Very low (less than 6 months)
- 40: Low (6 months - 1 year)
- 50: Average (1-2 years)
- 75: High (2-4 years)
- > 90: Very high (more than 4 years)

9. Career growth:
- < 15: No growth (stays at the same level)
- 25: Low (minor increase)
- 50: Average (moderate increase)
- 75: High (significant increase)
- > 90: Very high (rapid promotion up the career ladder)

10. Level of meticulousness of description:
- < 10: Very low (very few details and short)
- 25: Low (some details, but still short)
- 50: Average (fairly detailed, but not too detailed)
- 75: High (detailed and well described)
- > 90: Very high (extremely detailed description)

11. Cleverness level:
- < 20: Very low (very simple words with mistakes)
- 35: Low (simple words without mistakes)
- 50: Average (common words)
- 65: High (complex words)
- > 80: Very high (very complex and rare words)

12. Arrogance level:
- < 10: Very low (very modest)
- 20: Low (modest)
- 50: Average (a little confident, but not arrogant)
- 80: High (very confident, with elements of arrogance)
- > 90: Very high (very arrogant)

13. Education:
- < 20: Primary, secondary or no
- 40: Incomplete or specialized secondary education
- 60: Bachelor's degree
- 70: Master's degree
- > 85: Postgraduate or PhD

14. Academic status:
- < 30: Work experience only
- 50: Work experience own projects
- 65: Work experience, own projects, diploma in the specialty
- 80: Own projects, diploma in the specialty, publications
- > 90: Diploma in the specialty, publications, teaching

15. Foreign languages:
- < 10: No
- < 40: Basic English
- 60: Good English
- 75: Advanced English or several languages
- > 85: Resume in English

16. Professional certificates:
- < 25: No
- 40: There is a course
- 65: There are several courses
- 75: Many courses or scheduled certificates
- > 90: Many certificates and courses
"""

scale = """
1. Work experience:
- < 20: Less than a year
- > 90: 10+ years

2. Responsibility:
- < 20: Very low (cannot delegate tasks)
- < 40: Low (needs strong supervision)
- > 75: High (sometimes better to check)
- > 90: Very high (can be relied upon)

3. Creativity:
- < 20: Very low (follows instructions)
- > 90: Very high (constantly suggests innovative solutions)

4. Communication skills:
- 25: Very low (difficult to communicate)
- > 90: Very high (exceptional communication skills)

5. Leadership skills:
- < 15: Very low (never takes leadership)
- > 80: Very high (constantly shows leadership)

6. Stress resistance:
- < 25: Very low (cannot cope with stress)
- > 80: Very high (exceptional coping with stress)

7. Job difficulty:
- < 15: Very low (easy job, requiring no skills)
- > 90: Very high (expert level, complex tasks)

8. Average length of employment in one place:
- < 30: Very low (less than 6 months)
- > 90: Very high (more than 4 years)

9. Career growth:
- < 15: No growth (stays at the same level)
- > 90: Very high (rapid promotion up the career ladder)

10. Level of meticulousness of description:
- < 10: Very low (very few details and short)
- > 90: Very high (extremely detailed description)

11. Cleverness level:
- < 20: Very low (very simple words with mistakes)
- > 80: Very high (very complex and rare words)

12. Arrogance level:
- < 10: Very low (very modest)
- > 90: Very high (very arrogant)

13. Education:
- < 20: Primary, secondary or no
- > 85: Postgraduate or PhD

14. Academic status:
- < 30: Work experience only
- > 90: Diploma in the specialty, publications, teaching

15. Foreign languages:
- < 10: No
- > 85: Resume in English

16. Professional certificates:
- < 25: No
- > 90: Many certificates and courses
"""

In [174]:
class CV(BaseModel):
    work_experience: int
    responsibility: int
    creativity: int
    communication_skills: int
    leadership_qualities: int
    stress_resistance: int
    job_difficulty_level: int
    average_length_of_employment_at_one_job: int
    career_growth_speed: int
    description_details_level: int
    description_complexity_level: int
    arrogance_level: int
    education_quality: int
    academic_level: int
    foreign_languages: int
    professional_certificates: int


@sgl.function
def cv_parsing_gen(s, prompt):
    s += prompt + "\n\n" if prompt else ""
    s += "Give me a description about a provided CV in the JSON format. All values must be in percentiles between 0% and 100% of all other people.\n" + "Criteria for evaluation:\n" + scale + "\n"
    s += sgl.gen(
        "json_output",
        max_tokens=512,
        temperature=0,
        regex=build_regex_from_object(CV),
    )


def driver_cv_parsing_gen(prompt: str = ""):
    state = cv_parsing_gen.run(prompt)
    return state.text()

In [168]:
cv_path = glob('../data/cv/*')[1]

success, cv = extract_text_from_pdf(cv_path, _bytes=False)

In [169]:
print(cv)

Алёна Ильинична Соловьева
1
Алёна Ильинична Соловьева
Психолог
Москва
23 года
+7(929)-909-35-22
solovevaalena247@gmail.com
Психолог с опытом консультирования, ведения лекций и тренингов. Училась по 
обмену в Италии, диплом Специалиста и несколько курсов доп. образования. 
Суммарный опыт работы около двух лет. Увлекаюсь дизайном, вела медиаресурс. 
Основной приоритет - интересные проекты, в которых получится применить свои 
знания и навыки.
Опыт работы
Психолог в социальной 
сфере, ГБУ Мой 
семейный центр 
“Горизонт”,
мойсемейныйцентр.москва
июнь 2023 - ноябрь 2023
Индивидуальная и групповая психологическая 
работа с подростками.
Работа с аддикциями.
Выездные обучающие мероприятия для 
повышения качества работы персонала других 
центров.
Ведение документации.
Менеджер по работе с 
клиентами, СТС 
Эвентим РУ, ООО 
parter.ru
август 2018 - декабрь 2019
Ведение клиентов (физ. и юр. лиц).
Составление документации.
Работа на массовых мероприятиях.
Решение проблемных вопросов в рамках 
меропри

In [170]:
system = """You are given a resume. Make a short summary of this resume.
Describe work experience, education, skills, qualities and additional information in the resume.\n"""

resp = gen(system + cv)

In [171]:
print(resp.choices[0].message.content)

Алёна Соловьева is a licensed psychologist with 2 years of professional experience. She specializes in individual and group therapy for adolescents, addiction counseling, and training for staff at various organizations. 

Her work experience includes:

* **Social work:**  Providing psychological services at the "Horizon" family center.
* **Event management:** Managing clients, organizing events, and resolving issues.
* **Client management:** Working with both physical and legal entities.

She holds a specialization in extreme psychology and has completed courses in cognitive-behavioral therapy, Gestalt therapy, and media management. 

**Key skills:**

* **Languages:** Russian (native), English (C1), Italian (A2)
* **Software:** Figma, Photoshop, Excel, Word
* **Other:**  Intercultural communication, teamwork, statistical analysis, presentation and training skills.

**Additional information:**

* She has experience in online marketing, specifically designing and promoting shoes for an o

In [172]:
resp = driver_cv_parsing_gen(resp.choices[0].message.content)

In [144]:
json.loads(resp.split('\n')[-1]) # Daniel

{'work_experience': 6,
 'responsibility': 60,
 'creativity': 80,
 'communication_skills': 90,
 'leadership_qualities': 70,
 'stress_resistance': 85,
 'job_difficulty_level': 70,
 'average_length_of_employment_at_one_job': 50,
 'career_growth_speed': 80,
 'description_details_level': 85,
 'description_complexity_level': 80,
 'arrogance_level': 20,
 'education_quality': 80,
 'academic_level': 80,
 'foreign_languages': 80,
 'professional_certificates': 70}

In [173]:
json.loads(resp.split('\n')[-1]) # Alena

{'work_experience': 100,
 'responsibility': 80,
 'creativity': 70,
 'communication_skills': 90,
 'leadership_qualities': 85,
 'stress_resistance': 95,
 'job_difficulty_level': 70,
 'average_length_of_employment_at_one_job': 60,
 'career_growth_speed': 80,
 'description_details_level': 90,
 'description_complexity_level': 85,
 'arrogance_level': 20,
 'education_quality': 90,
 'academic_level': 85,
 'foreign_languages': 95,
 'professional_certificates': 80}

In [167]:
json.loads(resp.split('\n')[-1]) # Edwin

{'work_experience': 10,
 'responsibility': 80,
 'creativity': 90,
 'communication_skills': 95,
 'leadership_qualities': 70,
 'stress_resistance': 85,
 'job_difficulty_level': 80,
 'average_length_of_employment_at_one_job': 50,
 'career_growth_speed': 90,
 'description_details_level': 95,
 'description_complexity_level': 85,
 'arrogance_level': 20,
 'education_quality': 80,
 'academic_level': 85,
 'foreign_languages': 80,
 'professional_certificates': 80}

In [196]:
class CV(BaseModel):
    work_experience_level: int
    # average_length_of_employment: int
    # creativity_level: int
    responsibility_level: int
    leadership_skills_level: int
    stress_resistance_quality: int
    average_job_difficulty_level: int
    career_growth_speed: int
    nerdiness_level: int
    arrogance_level: int
    education_quality: int
    academic_level: int

@sgl.function
def cv_parsing_gen(s, prompt, schema):
    s += prompt + "\n\n"
    s += "Result should be in the JSON format.\n"
    s += sgl.gen(
        "json_output",
        max_tokens=512,
        temperature=0,
        regex=build_regex_from_object(schema),
    )

def driver_cv_parsing_gen(prompt: str, cv: str, schema):
    input_txt = f"{prompt}\n\n{cv}"
    state = cv_parsing_gen.run(input_txt, schema)
    return state.text()

In [213]:
cv_path = glob('../data/cv/*')[0]

success, cv = extract_text_from_pdf(cv_path, _bytes=False)

In [214]:
description_prompt = "You are given a cv and a json schema. Describe the resume in the required format. All values should be between 0 and 5."

cv_described = driver_cv_parsing_gen(description_prompt, cv, CV)
cv_described = json.loads(cv_described.split('\n')[-1])

In [215]:
cv_described  # Edwin

{'work_experience_level': 4,
 'responsibility_level': 4,
 'leadership_skills_level': 3,
 'stress_resistance_quality': 3,
 'average_job_difficulty_level': 4,
 'career_growth_speed': 4,
 'nerdiness_level': 4,
 'arrogance_level': 1,
 'education_quality': 4,
 'academic_level': 4}

In [212]:
cv_described  # Alena

{'work_experience_level': 2,
 'responsibility_level': 3,
 'leadership_skills_level': 2,
 'stress_resistance_quality': 3,
 'average_job_difficulty_level': 3,
 'career_growth_speed': 2,
 'nerdiness_level': 3,
 'arrogance_level': 1,
 'education_quality': 3,
 'academic_level': 3}

In [187]:
cv

'Алёна Ильинична Соловьева\n1\nАлёна Ильинична Соловьева\nПсихолог\nМосква\n23 года\n+7(929)-909-35-22\nsolovevaalena247@gmail.com\nПсихолог с опытом консультирования, ведения лекций и тренингов. Училась по \nобмену в Италии, диплом Специалиста и несколько курсов доп. образования. \nСуммарный опыт работы около двух лет. Увлекаюсь дизайном, вела медиаресурс. \nОсновной приоритет - интересные проекты, в которых получится применить свои \nзнания и навыки.\nОпыт работы\nПсихолог в социальной \nсфере, ГБУ Мой \nсемейный центр \n“Горизонт”,\nмойсемейныйцентр.москва\nиюнь 2023 - ноябрь 2023\nИндивидуальная и групповая психологическая \nработа с подростками.\nРабота с аддикциями.\nВыездные обучающие мероприятия для \nповышения качества работы персонала других \nцентров.\nВедение документации.\nМенеджер по работе с \nклиентами, СТС \nЭвентим РУ, ООО \nparter.ru\nавгуст 2018 - декабрь 2019\nВедение клиентов (физ. и юр. лиц).\nСоставление документации.\nРабота на массовых мероприятиях.\nРешение п